In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install wandb==0.16.0

In [ ]:
!pip install --upgrade datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("nateraw/pascal-voc-2012",split = 'train')

In [ ]:
# 데이터셋의 기본 정보 출력
print("데이터셋 정보:")
print(dataset)

# 데이터셋의 첫 번째 샘플 출력
print("\n첫 번째 데이터 샘플:")
print(dataset[0])

# 데이터셋의 특정 필드 이름 및 데이터 타입 탐색
print("\n필드 이름 및 데이터 타입:")
for field, field_type in dataset.features.items():
    print(f"{field}: {field_type}")

In [ ]:
import matplotlib.pyplot as plt

# 첫 번째 데이터 샘플을 로드
first_sample = dataset[0]
image = first_sample['image']
mask = first_sample['mask']

# 이미지와 마스크 표시
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# 원본 이미지
ax[0].imshow(image)
ax[0].set_title('original')
ax[0].axis('off')

# 마스크 이미지
ax[1].imshow(mask)
ax[1].set_title('mask')
ax[1].axis('off')

plt.show()


In [ ]:
import os
import math
import numpy as np
import tensorflow as tf
import random
import pickle

from PIL import Image
import matplotlib.pyplot as plt
from skimage.io import imread, imshow,show
from skimage.transform import resize
from glob import glob

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

In [ ]:
# random seed 설정
tf.random.set_seed(42)
random.seed(1234)

In [ ]:
image_width, image_height = 224, 224

from albumentations import  HorizontalFlip, RandomSizedCrop, Compose, OneOf, Resize

def build_augmentation(is_train=True):
  if is_train:    # 훈련용 데이터일 경우
    return Compose([
                    HorizontalFlip(p=0.5),    # 50%의 확률로 좌우대칭
                    RandomSizedCrop(         # 50%의 확률로 RandomSizedCrop
                        min_max_height=(300, 370),
                        w2h_ratio=370/1242,
                        height=image_height,
                        width=image_width,
                        p=0.5
                        ),
                    Resize(              # 입력이미지를 224X224로 resize
                        width=image_width,
                        height=image_height
                        )
                    ])
  return Compose([      # 테스트용 데이터일 경우에는 224X224로 resize만 수행합니다. 
                Resize(
                    width=image_width,
                    height=image_height
                    )
                ])

In [ ]:
from PIL import Image
import numpy as np

def resize_image(image, target_size=(224, 224), is_mask=False):
    """이미지를 지정된 크기로 리사이징하는 함수"""
    if is_mask:
        # 마스크의 경우 nearest neighbor 보간 사용
        return image.resize(target_size, Image.NEAREST)
    else:
        # 일반 이미지의 경우 기본 보간 방법 사용
        return image.resize(target_size)

# 이미지와 마스크를 NumPy 배열로 변환
input_images = np.array([np.array(resize_image(img)) for img in dataset['image']])
input_labels = np.array([np.array(resize_image(mask, is_mask=True)) for mask in dataset['mask']])

# 변환된 배열의 형태 확인
print("Images shape:", input_images.shape)
print("Labels shape:", input_labels.shape)



In [ ]:
# 데이터 확인
import numpy as np

augmentation_train = build_augmentation()
augmentation_test = build_augmentation(is_train=False)

# 훈련 데이터셋에서 5개만 가져와 augmentation을 적용해 봅시다.  
plt.figure(figsize=(12, 20))
for i in range(5):
    image =  np.array(dataset[i]['image'])
    image_data = {"image":image}
    resized = augmentation_test(**image_data)
    processed = augmentation_train(**image_data)
    plt.subplot(5, 2, 2*i+1)
    plt.imshow(resized["image"])  # 왼쪽이 원본이미지
    plt.subplot(5, 2, 2*i+2)
    plt.imshow(processed["image"])  # 오른쪽이 augment된 이미지
  
plt.show()

In [ ]:
image = input_images[0]
label = input_labels[0]
print(image.shape, label.shape)

In [ ]:
data = {
  "image": image,
  "mask": mask,
  }
image_data = {"image":image,"mask":label}
resized = augmentation_test(**image_data)
# resized_label = augmentation_test(**label_data)
print(resized['image'].shape, resized['mask'].shape)

In [ ]:
import numpy as np
from PIL import Image

def extract_class_color_mapping(masks):
    """
    마스크 이미지들에서 클래스별 색상 매핑을 추출하는 함수
    :param masks: 마스크 이미지의 리스트
    :return: 클래스와 색상의 매핑 딕셔너리
    """
    color_mapping = {}
    class_id = 0

    for mask in masks:
        # 마스크 이미지를 NumPy 배열로 변환
        mask_np = np.array(mask)
        # 고유한 색상 찾기
        unique_colors = np.unique(mask_np.reshape(-1, mask_np.shape[2]), axis=0)

        for color in unique_colors:
            # 색상을 튜플로 변환
            color_tuple = tuple(color)
            # 새로운 색상이면 클래스 ID 할당
            if color_tuple not in color_mapping:
                color_mapping[color_tuple] = class_id
                class_id += 1

    return color_mapping



def map_color_mask_to_integer(color_mask, mapping):
    integer_mask = np.zeros(color_mask.shape[:-1], dtype=np.uint8)
    for row in range(color_mask.shape[0]):
        for col in range(color_mask.shape[1]):
            color = tuple(color_mask[row, col])
            if color in mapping:
                integer_mask[row, col] = mapping[color]
    return integer_mask

In [ ]:
# 데이터셋에서 마스크 목록 추출
masks = [sample['mask'] for sample in dataset]

# 클래스와 색상 매핑 추출
class_color_mapping = extract_class_color_mapping(masks)

# 매핑 결과 출력
print("클래스와 색상 매핑:")
print(class_color_mapping)

In [ ]:
def create_inverse_mapping(class_color_mapping):
    """
    클래스 ID에서 색상으로의 역 매핑을 생성하는 함수
    :param class_color_mapping: 클래스와 색상의 매핑 딕셔너리
    :return: 역 매핑 딕셔너리
    """
    inverse_mapping = {class_id: color for color, class_id in class_color_mapping.items()}
    return inverse_mapping

In [ ]:
inverse_mapping = create_inverse_mapping(class_color_mapping)
# 역 매핑 결과 출력
print("역 매핑:")
print(inverse_mapping)

In [ ]:
# 5행 5열의 서브플롯 생성
plt.figure(figsize=(15, 15))
for class_id, color in inverse_mapping.items():
    ax = plt.subplot(5, 5, class_id + 1)
    ax.set_title(f"Class {class_id}")
    ax.imshow(np.full((10, 10, 3), color, dtype=np.uint8))  # 색상으로 채운 작은 이미지 생성
    ax.axis('off')

# 나머지 빈 서브플롯 숨기기
for i in range(len(inverse_mapping) + 1, 26):
    ax = plt.subplot(5, 5, i)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 컬러 시맨틱 세그멘테이션 마스크를 정수로 변환
integer_mask = map_color_mask_to_integer(np.array(resized['mask']), class_color_mapping)
integer_mask.shape

In [ ]:
def map_integer_mask_to_color(integer_mask, reverse_mapping):
    color_mask = np.zeros((integer_mask.shape[0], integer_mask.shape[1], 3), dtype=np.uint8)
    for row in range(integer_mask.shape[0]):
        for col in range(integer_mask.shape[1]):
            class_idx = integer_mask[row, col]
            color = reverse_mapping[class_idx]
            # 컬러 값을 [0, 255] 범위로 스케일링
            scaled_color = np.clip(color, 0, 255)
            color_mask[row, col] = scaled_color

    return color_mask

restored_color_mask = map_integer_mask_to_color(integer_mask, inverse_mapping)

In [ ]:
plt.figure(figsize = (18,5))
plt.subplot(1, 3, 1)
plt.imshow(resized["image"])
plt.subplot(1, 3, 2)
plt.imshow(integer_mask)
plt.subplot(1, 3, 3)
plt.imshow(Image.blend(Image.fromarray(resized["image"]),Image.fromarray(restored_color_mask), alpha=0.5))
plt.show()

In [ ]:
import os
import math
import random
import numpy as np
from glob import glob
from skimage.io import imread
from PIL import Image
from datasets import DatasetDict
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self , batch_size=16, img_size=(224, 224, 3), output_size=(224, 224),
                 is_train=True, is_val=False, augmentation=None):
        '''
        batch_size: batch_size입니다.
        img_size: preprocess에 사용할 입력이미지의 크기입니다.
        output_size: ground_truth를 만들어주기 위한 크기입니다.
        is_train: 이 Generator가 학습용인지 테스트용인지 구분합니다.
        augmentation: 적용하길 원하는 augmentation 함수를 인자로 받습니다.
        '''
        self.batch_size = batch_size
        self.is_train = is_train
        self.is_val = is_val
        self.augmentation = augmentation
        self.img_size = img_size
        self.output_size = output_size

        self.data = self.load_dataset().to_list()
        print(type(self.data))

    def load_dataset(self):
        dataset = load_dataset("nateraw/pascal-voc-2012", split='train')
        shuffled_dataset = dataset.shuffle(seed=42)

        # 데이터셋 분할 비율 설정
        train_ratio = 0.7
        val_ratio = 0.2
        test_ratio = 0.1

        # 훈련, 검증, 테스트 세트로 분할
        split_datasets = shuffled_dataset.train_test_split(test_size=(val_ratio + test_ratio))
        test_dataset = split_datasets['test'].train_test_split(test_size=test_ratio / (val_ratio + test_ratio))

        # 최종적으로 분할된 데이터셋
        data = DatasetDict({
            'train': split_datasets['train'],
            'validation': test_dataset['train'],
            'test': test_dataset['test']
        })

        if self.is_train:
            return data['train']
        elif self.is_val:
            return data['validation']
        else:
            return data['test']
    def __len__(self):
        return math.ceil(len(self.data) / self.batch_size)

    def __getitem__(self, index):
        print(self.data)
        batch_data = self.data[index*self.batch_size : (index + 1)*self.batch_size]
        print(type(batch_data))
        inputs = np.zeros([self.batch_size, *self.img_size])
        outputs = np.zeros([self.batch_size, *self.output_size[:2]])
        for i, data in enumerate(batch_data):
            print(type(data))
            img, mask = data
            
            _input = np.array(resize_image(img))
            _output = np.array(resize_image(mask))


            augmented = self.augmentation(image=_input, mask=mask) if self.augmentation else {'image': _input, 'mask': _output}

            integer_mask = map_color_mask_to_integer(augmented['mask'])
            outputs[i] = integer_mask

            image_array = augmented["image"] / 255.0
            inputs[i] = image_array

        return inputs, outputs



    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.is_train:
            np.random.shuffle(self.indexes)
        return self.indexes


In [ ]:
augmentation = build_augmentation()
test_preproc = build_augmentation(is_train=False)
        
train_generator = DataGenerator(
    augmentation=augmentation,
)
val_generator = DataGenerator(
    augmentation=test_preproc,
    is_train = False,
    is_val = True
)
test_generator = DataGenerator(
    augmentation=test_preproc,
    is_train = False,
    is_val = False
)

In [ ]:
train_generator.batch_size,val_generator.batch_size,test_generator.batch_size

In [ ]:
len(train_generator),len(val_generator),len(test_generator) # *16 해야 데이터 수 근사

In [ ]:
for image, mask in train_generator:
        sample_image, sample_mask = image, mask
        break
sample_image.shape, sample_mask.shape